In [2]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()

    import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

In [2]:
BASE = "https://www.mncourts.gov"
start = 'https://www.mncourts.gov/getforms.aspx'
connection = requests.get(start)
html = connection.text
soup = bs4.BeautifulSoup(html,"lxml")
connection.close()

In [ ]:
BASE = "https://www.mncourts.gov"
ul = soup.find("ul", class_ = "formCategoryList")
a = ul.find_all('a')
form_categories = {}
for refs in a:
    form_categories[refs.text]= BASE+refs['href']
print(form_categories)
form = form_categories['Adoption']
soup = connect_to_page(form)

In [ ]:
forms_within_cat = []
ahref = soup.find_all('a',href = True)
for i in ahref:
    if '/GetForms.aspx?' in i['href']:
        forms_within_cat.append(BASE+i['href'])
print(forms_within_cat)
soup = connect_to_page(form)

MN FORMS EXPLORER (0 FAULTY DOWNLOADS)

In [ ]:

BASE = "https://www.mncourts.gov"
source = 'https://www.mncourts.gov/getforms.aspx'
today = date.today().strftime("%Y-%m-%d")
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded",])
form_dest = "PDFS/MN_forms/All_forms_testingDF/"
jur = 'MN'
count = 0

#---------------------------------FIRST CONNECTION - CATEGORIES-----------------------------------------------------------
soup = connect_to_page(source)
ul = soup.find("ul", class_ = "formCategoryList")
a = ul.find_all('a')
form_categories = {}
for refs in a:
    form_categories[refs.text]= BASE+refs['href']


pdfs = {}
sub_categories = []


#---------------------------------SECOND CONNECTION - SUBCATEGORIES-----------------------------------------------------------
for key,value in form_categories.items():
    sub_categories = []
    # time.sleep(2)
    sub_cat = value
    curr_group = key
    soup = connect_to_page(sub_cat)
    ahref = soup.find_all('a',href = True)
    for i in ahref:
        if '/GetForms.aspx?' in i['href']:
            sub_categories.append(BASE+i['href'])

    print('--------------------SUBCATEGORY--------------------------')
    print(key)

    #---------------------------------DEALING WITH EXCEPTION-----------------------------------------------------------
    #only two cases in which the format changes. We only deal with the first. The second page just redirects you to source
    if len(sub_categories) == 0 and count==0:
        count+=1
        exception_refs = soup.find_all('a', href=True)
        for link in exception_refs:
            if '.pdf' in link['href']:
                url = BASE+link['href']
                title = link.text
                filename = link['href'].split('/')[-1]
                title = re.sub(r"\([^()]*\)","",title)
                print(filename,title)

                if filename not in pdfs:
                    fileid = hashme(url)
                    # download_pdf_unmasked(url, form_dest,fileid+'.pdf')
                    pdfs[filename] = title
                    # if status=='error':
                    #     print("##########################")
                    #     print('ERROR',filename,url)
                    #     print("##########################")
                    files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,curr_group,url,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]))


    #---------------------------------FORMS FROM EACH SUBCATEGORY-----------------------------------------------------------
    for form in sub_categories:
        time.sleep(1)
        soup = connect_to_page(form)
        # print(form)
        href = soup.find_all('a',href= True)
        for pdf in href:
            
            if '.pdf' in pdf['href'] and 'www' not in pdf['href'] and 'http' not in pdf['href']:
                print(pdf['href'])
                url = BASE+pdf['href']
                title = pdf.get("title")
                if title!=None:
                    title = title[:len(title)-4] #last 4 chars is simple 'PDF'
                else:
                    title = pdf.text
                title = re.sub(r"\([^()]*\)","",title)
                filename = pdf['href'].split('/')[-1].split('?')[0]
                if filename not in pdfs:
                    print(filename,title)
                    fileid = hashme(url)
                    download_pdf_unmasked(url, form_dest,fileid+'.pdf')
                    pdfs[filename] = title 
                    # if status=='error':
                    #     print("##########################")
                    #     print('ERROR',filename,url)
                    #     print("##########################")
                    files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,curr_group,url,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]))
files_df.to_csv("MN-V2.csv",index=False, encoding="utf-8")

In [ ]:
BASE = "https://www.mncourts.gov"
soup = connect_to_page("https://www.mncourts.gov/GetForms.aspx?c=4&p=5")

href = soup.find_all('a',href= True)
for pdf in href:  
    if '.pdf' in pdf['href']:
        print(pdf['href']) 
        url = BASE+pdf['href']
        title = pdf.get("title")
        
        if title!= None:
            title = title[:len(title)-4] #last 3 chars is simple 'PDF'
        else:
            title = pdf.text
        title = re.sub(r"\([^()]*\)","",title)
        print(title)